<div style="display: flex; align-items: flex-start;">
  <div style="flex: 1; max-width: 20%; padding-right: 10px;">
    <img src="./images/RAGme-logo.png" width=150px>
  </div>
  <div style="flex: 2; max-width: 80%;">
    <center><h1>RAGme</h1></center>
    <p>The RAGme project is an innovative platform designed to enhance information retrieval and response generation. It combines the power of Retrieval-Augmented Generation (RAG) with a robust, vectorized database, allowing users to upload their knowledge base documents which are then transformed into a searchable format. This setup enables the system to efficiently retrieve relevant information in response to user queries.</p>
  </div>
</div>



# Imports

Import all the necessary libraries to launch ChatLLM

In [ ]:
import os
from dotenv import load_dotenv

import gradio as gr
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.vectorstores import FAISS
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

# Environment Variables and API Validation

Starts loading environment variables from a `.env` file inside the project folder

Create the `.env` file with the following cell, open it and store inside it your API keys:
<br>
<br>➡️ OPENAI_API_KEY has to be setted with your OpenAI API key created from https://platform.openai.com/account

In [83]:
! [ ! -f .env ] && touch .env && echo "OPENAI_API_KEY=your_openai_api_key_here"> .env >> .env

Load the environment variables from the `.env` file

In [84]:
print("⏳ Loading environment variables...")

# load environment variables
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
ollama_api_key = os.getenv('OLLAMA_API_KEY')

# check OpenAI API key
if not openai_api_key or not openai_api_key.startswith("sk-proj-"):
    print("❌ ERROR: OpenAI API key invalid or missing.")
else:
    print("✅ OpenAI API key found and valid.")

⏳ Loading environment variables...
✅ OpenAI API key found and valid.


# Folders and Model initialization

Define the `upload` folder and the `vector_db` folder where the input files and the corresponding vectorized database will take place

In [87]:
# directory where uploaded files will be stored
UPLOAD_DIR = "upload"
VECTOR_DB_PATH = "vector_db"

# create the upload directory if it doesn't exist
if not os.path.exists(UPLOAD_DIR):
    os.makedirs(UPLOAD_DIR)
    print("✅ Upload directory created.")
else:
    print("✅ Upload directory already exists.")


# create the vector database directory if it doesn't exist
if not os.path.exists(VECTOR_DB_PATH):
    os.makedirs(VECTOR_DB_PATH)
    print("✅ Vector database directory created.")
else:
    print("✅ Vector database directory already exists.")

✅ Upload directory already exists.
✅ Vector database directory already exists.


Choose your LLM model:

In [90]:
MODEL="gpt-4o-mini"

print(f"✅ Model choosed: {MODEL}")

✅ Model choosed: gpt-4o-mini


# Create the database

Create the vectorized database with FAISS standard.

In [ ]:
# function to process all files in UPLOAD_DIR and create a vectorized database
def process_all_files():
    documents = []

    if os.listdir(UPLOAD_DIR) == []:
        return f"❌ Missing files in the upload directory"
    
    for file_name in os.listdir(UPLOAD_DIR):
        file_path = os.path.join(UPLOAD_DIR, file_name)
        
        if file_path.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif file_path.endswith(".txt"):
            loader = TextLoader(file_path)
        else:
            continue  # skip unsupported file types
        
        documents.extend(loader.load())
    
    # split documents into smaller chunks
    text_splitter = CharacterTextSplitter(separator="\n\n", chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_documents(documents)
    
    # create vectorized embeddings
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(chunks, embedding=embeddings)
    vectorstore.save_local(VECTOR_DB_PATH)

    return f"✅ Processed {len(chunks)} document chunks and stored in FAISS database."

# Inizialize the RAG pipleine

In [92]:
# function to initialize the RAG model
def initialize_rag():
    if not os.path.exists(VECTOR_DB_PATH):
        return "❌ No vector database found. Upload and process files first."
    
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.load_local(VECTOR_DB_PATH, embeddings, allow_dangerous_deserialization=True)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    llm = ChatOpenAI(temperature=0.7, model_name=MODEL, streaming=True)
    
    return ConversationalRetrievalChain.from_llm(llm=llm, retriever=retriever, memory=memory)

In [93]:
# function to handle chat queries
def chat_rag(query, history):
    global conversation_chain
    if conversation_chain is None:
        conversation_chain = initialize_rag()
        if isinstance(conversation_chain, str):
            return conversation_chain  # return error message if vector DB is missing
    
    result = conversation_chain.invoke({"question": query})
    return result["answer"]

# Launch the application

Launch the gradio UI interface on the RAG pipeline.

In [102]:
# create the Gradio interface
with gr.Blocks() as app:
    # embed the chat interface inside the Blocks UI
    chat_ui = gr.ChatInterface(chat_rag, title="RAGme", description="1. Move your knowledge base documents in `upload`\n2. Click on `Build Knowledge Base`\n3. Ask to GPT")

    # build the vector database
    process_button = gr.Button("Build Knowledge Base")
    process_output = gr.Textbox(label="Status:", placeholder="Load your documents in the upload directory and click the 'Build Knowledge Base' button.")
    process_button.click(process_all_files, outputs=[process_output])

# launch the Gradio app
app.launch(inbrowser=True)


/Users/giuliorusso/miniconda3/envs/llm/lib/python3.13/site-packages/gradio/components/chatbot.py:288: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7903

To create a public link, set `share=True` in `launch()`.
